## Library

In [14]:
import requests
import random
import pandas as pd
import time
import math
from copy import deepcopy
from statistics import mean

## Notebook

#### Requisição

In [6]:
def request() -> list:   
    res = requests.get('http://localhost:3000/api/ep1')
    return res.json()

#### Tratamento de Dados

In [147]:
def data_treatment(json:list):
    for i in range(len(json)):
        for key in json[i]:
            if ('prod' in key):
                if (type(json[i][key])!=dict):
                    qnt = json[i][key]
                    if (type(qnt)==str):
                        qnt = float(qnt)                  
                    if (qnt<0):
                        qnt = 0
                    if (key in [product for product in json[i] if ('prod' in product and (int(product[5]) < 9 and len(product) < 7))]):
                        json[i][key] = {'qnt':[round(qnt)], 'price':None}
                    else:
                        json[i][key] = {'qnt':[qnt], 'price':None}
        try:
            json[i]['date'] = time.ctime(json[i]['date'])
        except:
            continue
    return json

#### Gerar valores aleatórios

In [161]:
def generate_random_price(n_min=1.0, n_max=100.0) -> float:
    return round(random.uniform(n_min, n_max), 2)

#### Precificação

##### Função logistica -> A mesma postada no doc pelo professor

In [9]:
def logistic_function(v: float) -> float:
    return 0.5 + (1 / 1 + math.e)**(-1 * v)

##### Função para calcular a variação -> A mesma postada no doc pelo professor

In [10]:
def calc_qnt_variation(last_week: list, current_week: list) -> float:
    return mean(current_week) - mean(last_week) / mean(last_week) if (mean(last_week) != 0) else 0

##### Função que calcula o preço que será utilizado na semana seguinte

In [11]:
def calc_new_price(product_db: dict, sales_week: dict) -> dict:
    for product in product_db:
        if (product_db[product]['qnt'] != None):
            if (product not in sales_week.keys()):
                sales_week[product] = [0]
            product_db[product]['price'] = logistic_function(calc_qnt_variation(product_db[product]['qnt'], sales_week[product])) * product_db[product]['price']
        product_db[product]['qnt'] = sales_week[product]               
    return product_db

##### Função que atualiza o banco de dados (onde consta a quantidade de vendas da última semana e o preço dos produtos) e aplica o preço nos produtos do JSON

In [12]:
def product_pricing(json: list, product_db: dict) -> tuple:
    sales_week = {}

    for transaction in json:
        for key in transaction:
            if ('prod' in key):
                if (key not in product_db.keys()):
                    transaction[key]['price'] = generate_random_price()
                    sales_week[key] = deepcopy(transaction[key]['qnt'])
                    product_db[key] = deepcopy(transaction[key])
                    product_db[key]['qnt'] = None              
                else:
                    transaction[key]['price'] = deepcopy(product_db[key]['price'])
                    if (key not in sales_week.keys()):
                        sales_week[key] = deepcopy(transaction[key]['qnt'])
                    else:
                        sales_week[key] += deepcopy(transaction[key]['qnt'])

    product_db = calc_new_price(product_db, sales_week)

    return json, product_db

#### Main

In [181]:
product_database = {}
weeks = 25

for week in range(weeks):
    json = data_treatment(request())
    json, product_database = product_pricing(json, product_database)

    json_db = pd.DataFrame(json)
    product_database_db = pd.DataFrame(product_database)

    print(f'\nSemana {week}:\n')
    print('Json:')
    display(json_db)
    print('\nproduct_database:')
    display(product_database_db)


Semana 0:

Json:


,id,date,prod_4,prod_5,prod_6,prod_7,prod_8,prod_9,prod_11,prod_15,prod_0,prod_3,prod_2,prod_1,prod_10
0,labakh2jkmry3apkmk,Wed Oct 14 19:46:28 2026,"{'qnt': [50], 'price': 5.86}","{'qnt': [8], 'price': 7.55}","{'qnt': [49], 'price': 42.07}","{'qnt': [15], 'price': 45.47}","{'qnt': [130], 'price': 91.8}","{'qnt': [178.27969565571377], 'price': 85.46}","{'qnt': [707.2929457882833], 'price': 5.05}","{'qnt': [2403.800804321996], 'price': 84.36}",NaN,NaN,NaN,NaN,NaN
1,labakh2jmcrsqgtkxrr,Thu Oct 15 03:45:55 2026,NaN,NaN,NaN,NaN,"{'qnt': [724], 'price': 91.8}","{'qnt': [17.733095963655433], 'price': 85.46}","{'qnt': [549.8175539847741], 'price': 5.05}","{'qnt': [2065.84290818358], 'price': 84.36}","{'qnt': [27], 'price': 50.12}","{'qnt': [8], 'price': 92.5}",NaN,NaN,NaN
2,labakh2jefaqmlecjrr,Thu Oct 15 06:25:14 2026,"{'qnt': [50], 'price': 5.86}",NaN,NaN,NaN,"{'qnt': [8], 'price': 91.8}","{'qnt': [405.5526630923876], 'price': 85.46}","{'qnt': [35.9745885679758], 'price': 5.05}","{'qnt': [1802.3292691696788], 'price': 84.36}",NaN,NaN,"{'qnt': [20], 'price': 79.35}",NaN,NaN
3,labakh2jc2a5vdzdkne,Thu Oct 15 00:48:56 2026,"{'qnt': [31], 'price': 5.86}",NaN,"{'qnt': [3], 'price': 42.07}",NaN,NaN,"{'qnt': [34.93275697749383], 'price': 85.46}",NaN,"{'qnt': [1979.2615483441873], 'price': 84.36}","{'qnt': [27], 'price': 50.12}","{'qnt': [2], 'price': 92.5}","{'qnt': [24], 'price': 79.35}","{'qnt': [45], 'price': 14.48}","{'qnt': [399.72270008206795], 'price': 67.81}"
4,labakh2j9h71ppexs9c,Thu Oct 15 00:09:47 2026,NaN,NaN,"{'qnt': [8], 'price': 42.07}",NaN,"{'qnt': [17], 'price': 91.8}","{'qnt': [392.4723498240192], 'price': 85.46}","{'qnt': [132.36719850309927], 'price': 5.05}","{'qnt': [2185.690462807609], 'price': 84.36}",NaN,"{'qnt': [43], 'price': 92.5}",NaN,"{'qnt': [5], 'price': 14.48}",NaN
5,labakh2jbuionwk0cis,Wed Oct 14 19:44:11 2026,"{'qnt': [27], 'price': 5.86}",NaN,"{'qnt': [25], 'price': 42.07}","{'qnt': [38], 'price': 45.47}","{'qnt': [33], 'price': 91.8}","{'qnt': [2.2141439484628767], 'price': 85.46}","{'qnt': [731.3714870170469], 'price': 5.05}","{'qnt': [1812.4081934149021], 'price': 84.36}",NaN,NaN,NaN,"{'qnt': [25], 'price': 14.48}",NaN
6,labakh2jvrnb5c2y9z,Wed Oct 14 19:42:06 2026,"{'qnt': [37], 'price': 5.86}","{'qnt': [6], 'price': 7.55}",NaN,NaN,"{'qnt': [33], 'price': 91.8}","{'qnt': [30.693875789100034], 'price': 85.46}","{'qnt': [30.78543926423052], 'price': 5.05}","{'qnt': [2418.859969914639], 'price': 84.36}","{'qnt': [14], 'price': 50.12}","{'qnt': [12], 'price': 92.5}",NaN,NaN,"{'qnt': [428.6980956064099], 'price': 67.81}"
7,labakh2j0e7vwdwl6l8k,Wed Oct 14 19:14:18 2026,NaN,"{'qnt': [5], 'price': 7.55}","{'qnt': [3], 'price': 42.07}",NaN,"{'qnt': [549], 'price': 91.8}",NaN,"{'qnt': [552.4208910714818], 'price': 5.05}","{'qnt': [1995.9924601481011], 'price': 84.36}","{'qnt': [34], 'price': 50.12}","{'qnt': [28], 'price': 92.5}","{'qnt': [25], 'price': 79.35}","{'qnt': [24], 'price': 14.48}",NaN
8,labakh2jcgpx345q8d8,Thu Oct 15 03:07:01 2026,NaN,NaN,"{'qnt': [42], 'price': 42.07}",NaN,"{'qnt': [42], 'price': 91.8}","{'qnt': [23.466882695363957], 'price': 85.46}","{'qnt': [123.89909774888262], 'price': 5.05}","{'qnt': [1929.8112624041771], 'price': 84.36}",NaN,"{'qnt': [26], 'price': 92.5}","{'qnt': [34], 'price': 79.35}",NaN,NaN
9,labakh2j87orb55ufzu,Wed Oct 14 17:45:34 2026,NaN,NaN,"{'qnt': [31], 'price': 42.07}",NaN,"{'qnt': [345], 'price': 91.8}","{'qnt': [41.69006120013157], 'price': 85.46}","{'qnt': [278.36660725966675], 'price': 5.05}","{'qnt': [1909.113474770841], 'price': 84.36}","{'qnt': [32], 'price': 50.12}",NaN,"{'qnt': [41], 'price': 79.35}",NaN,"{'qnt': [441.83137912890135], 'price': 67.81}"



product_database:


,prod_4,prod_5,prod_6,prod_7,prod_8,prod_9,prod_11,prod_15,prod_0,prod_3,prod_2,prod_1,prod_10
qnt,"[50, 50, 31, 27, 37, 17, 22, 41, 49, 44, 45, 4...","[8, 6, 5, 14, 6, 45, 37, 10, 41, 34]","[49, 3, 8, 25, 3, 42, 31, 30, 39, 12, 25, 2, 4...","[15, 38, 0, 41, 6, 8, 18, 46, 41, 12, 50, 46, ...","[130, 724, 8, 17, 33, 33, 549, 42, 345, 20, 43...","[178.27969565571377, 17.733095963655433, 405.5...","[707.2929457882833, 549.8175539847741, 35.9745...","[2403.800804321996, 2065.84290818358, 1802.329...","[27, 27, 14, 34, 32, 34, 6, 35, 6, 15, 38, 42,...","[8, 2, 43, 12, 28, 26, 26, 49, 50, 31, 5, 29, ...","[20, 24, 25, 34, 41, 29, 45, 2, 10, 19, 9, 14,...","[45, 5, 25, 24, 34, 3, 9, 27, 3, 36, 11, 27, 26]","[399.72270008206795, 428.6980956064099, 441.83..."
price,5.86,7.55,42.07,45.47,91.8,85.46,5.05,84.36,50.12,92.5,79.35,14.48,67.81



Semana 1:

Json:


...

,prod_4,prod_5,prod_6,prod_7,prod_8,prod_9,prod_11,prod_15,prod_0,prod_3,prod_2,prod_1,prod_10
qnt,"[32, 34, 14, 12]","[32, 30, 12, 47]","[36, 25, 15]","[49, 47]","[0, 38, 115, 153, 150, 0, 0]","[0, 29.901272611210405, 370.8124986777665, 5.3...","[0, 163.48001819810656, 15.149116491809991, 40...","[2061.452166211052, 2045.2403348202154, 2102.0...","[48, 43, 7, 39]","[23, 4, 27, 5, 14]","[35, 5, 10, 33, 2]","[29, 28, 20]","[0, 36.35373945920446, 475.21624347252526, 0, ..."
price,0.000053,0.000577,0.000127,0.000137,0.000011,0.00001,0.000386,0.00001,0.000151,0.000011,0.00024,0.000044,0.000008



Semana 24:

Json:


,id,date,prod_4,prod_6,prod_9,prod_8,prod_15,prod_0,prod_1,prod_2,prod_11,prod_7,prod_10,prod_3,prod_5
0,labakiqa6ifvdt78wr5,Wed Mar 31 23:24:52 2027,"{'qnt': [19], 'price': 5.304150306383263e-05}","{'qnt': [6], 'price': 0.0001269315170619199}","{'qnt': [114.62895345127356], 'price': 1.01876...","{'qnt': [482], 'price': 1.0943412785170957e-05}","{'qnt': [2359.503733623845], 'price': 1.005649...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,labakiqafk184flfxlb,Thu Apr 1 08:09:18 2027,NaN,NaN,"{'qnt': [464.8031273523941], 'price': 1.018762...","{'qnt': [509], 'price': 1.0943412785170957e-05}","{'qnt': [2317.645468270847], 'price': 1.005649...","{'qnt': [31], 'price': 0.00015121958375088245}","{'qnt': [34], 'price': 4.368837022599542e-05}","{'qnt': [7], 'price': 0.00023959262228445927}","{'qnt': [104.76041271454565], 'price': 0.00038...",NaN,NaN,NaN,NaN
2,labakiqa7hnhbtq02hd,Wed Mar 31 19:19:08 2027,NaN,NaN,"{'qnt': [724.5809977171391], 'price': 1.018762...","{'qnt': [31], 'price': 1.0943412785170957e-05}","{'qnt': [2004.8275727840974], 'price': 1.00564...",NaN,"{'qnt': [17], 'price': 4.368837022599542e-05}",NaN,NaN,"{'qnt': [16], 'price': 0.00013719184288806067}","{'qnt': [389.42751939774], 'price': 8.08358192...",NaN,NaN
3,labakiqa7gf389v1wgc,Wed Mar 31 23:37:46 2027,NaN,"{'qnt': [1], 'price': 0.0001269315170619199}","{'qnt': [541.1778949480708], 'price': 1.018762...",NaN,"{'qnt': [2493.5179407894184], 'price': 1.00564...",NaN,NaN,NaN,"{'qnt': [707.9923894615264], 'price': 0.000385...","{'qnt': [8], 'price': 0.00013719184288806067}","{'qnt': [110.82974271703912], 'price': 8.08358...","{'qnt': [27], 'price': 1.1059066439874732e-05}",NaN
4,labakiqao7axrgwkdx,Thu Apr 1 03:10:00 2027,NaN,NaN,"{'qnt': [83.40479142904297], 'price': 1.018762...","{'qnt': [355], 'price': 1.0943412785170957e-05}","{'qnt': [2187.603808825252], 'price': 1.005649...","{'qnt': [40], 'price': 0.00015121958375088245}","{'qnt': [2], 'price': 4.368837022599542e-05}","{'qnt': [1], 'price': 0.00023959262228445927}",NaN,"{'qnt': [31], 'price': 0.00013719184288806067}","{'qnt': [231.55078563378024], 'price': 8.08358...","{'qnt': [5], 'price': 1.1059066439874732e-05}","{'qnt': [22], 'price': 0.0005765419755841958}"
5,labakiqao7rfcwn9gqt,Thu Apr 1 00:46:28 2027,"{'qnt': [29], 'price': 5.304150306383263e-05}","{'qnt': [34], 'price': 0.0001269315170619199}","{'qnt': [35.92980197664567], 'price': 1.018762...",NaN,"{'qnt': [2043.8468063821722], 'price': 1.00564...",NaN,"{'qnt': [38], 'price': 4.368837022599542e-05}","{'qnt': [23], 'price': 0.00023959262228445927}",NaN,NaN,"{'qnt': [527.7718352088065], 'price': 8.083581...","{'qnt': [17], 'price': 1.1059066439874732e-05}","{'qnt': [23], 'price': 0.0005765419755841958}"
6,labakiqagu2wtxujhfv,Wed Mar 31 21:08:17 2027,NaN,NaN,"{'qnt': [8.584582315491451], 'price': 1.018762...","{'qnt': [374], 'price': 1.0943412785170957e-05}","{'qnt': [1924.120684597068], 'price': 1.005649...","{'qnt': [3], 'price': 0.00015121958375088245}","{'qnt': [22], 'price': 4.368837022599542e-05}","{'qnt': [6], 'price': 0.00023959262228445927}",NaN,NaN,"{'qnt': [280.19556322725947], 'price': 8.08358...","{'qnt': [50], 'price': 1.1059066439874732e-05}",NaN
7,labakiqa9d9wurp9uxk,Thu Apr 1 06:04:56 2027,NaN,NaN,NaN,"{'qnt': [5], 'price': 1.0943412785170957e-05}","{'qnt': [2103.1142760324287], 'price': 1.00564...","{'qnt': [7], 'price': 0.00015121958375088245}",NaN,"{'qnt': [6], 'price': 0.00023959262228445927}",NaN,NaN,NaN,NaN,"{'qnt': [10], 'price': 0.0005765419755841958}"
8,labakiqa9ex1dwe2zqf,Thu Apr 1 02:47:43 2027,"{'qnt': [29], 'price': 5.304150306383263e-05}",NaN,"{'qnt': [220.16893471921787], 'price': 1.01876...","{'qnt': [15], 'price': 1.0943412785170957e-05}","{'qnt': [1806.768349534966], 'price': 1.005649...","{'qnt': [10], 'price': 0.00015121958375088245}","{'qnt': [11], 'price': 4.368837022599542e-05}",NaN,"{'qnt': [175.4155391030215], 'price': 0.000385...","{'qnt': [6], 'price': 0.00013719184288806067}","{'qnt': [110.31334235702309], 'price': 8.08358.


product_database:


,prod_4,prod_5,prod_6,prod_7,prod_8,prod_9,prod_11,prod_15,prod_0,prod_3,prod_2,prod_1,prod_10
qnt,"[19, 29, 29, 33, 28, 22, 39, 8]","[22, 23, 10, 46, 14, 23, 6, 47, 21, 48, 48]","[6, 1, 34, 5, 20, 41, 14, 6, 37, 26]","[16, 8, 31, 6, 11, 19, 16, 8, 24]","[482, 509, 31, 355, 374, 5, 15, 353, 24, 381, ...","[114.62895345127356, 464.8031273523941, 724.58...","[104.76041271454565, 707.9923894615264, 175.41...","[2359.503733623845, 2317.645468270847, 2004.82...","[31, 40, 3, 7, 10, 46, 6, 32]","[27, 5, 17, 50, 37, 23, 9, 9, 23, 12, 2, 19]","[7, 1, 23, 6, 6, 12, 44, 8, 40, 47, 36]","[34, 17, 2, 38, 22, 11, 4, 31, 11, 36, 21]","[389.42751939774, 110.82974271703912, 231.5507..."
price,0.000027,0.000288,0.000063,0.000069,0.000005,0.000005,0.000193,0.000005,0.000076,0.000006,0.00012,0.000022,0.000004
